In [ ]:
from black_friday.data_access.blackfriday_data import BlackFridayData
from black_friday.entity.config_entity import DataIngestionConfig

data_ingestion_config=DataIngestionConfig()
 
data_ingestion_config = data_ingestion_config

usvisa_data = BlackFridayData()
dataframe = usvisa_data.export_collection_as_dataframe(collection_name=
                                                        data_ingestion_config.collection_name)


print(dataframe)


In [ ]:
from black_friday.configuration.mongo_db_connection import MongoDBClient
from black_friday.constants import DATABASE_NAME
from black_friday.exception import BlackFridayException
import pandas as pd
import sys
from typing import Optional
import numpy as np



class BlackFridayData:
    """
    This class help to export entire mongo db record as pandas dataframe
    """

    def __init__(self):
        """
        """
        try:
            self.mongo_client = MongoDBClient(database_name=DATABASE_NAME)
        except Exception as e:
            raise BlackFridayException(e,sys)
        

    def export_collection_as_dataframe(self,collection_name:str,database_name:Optional[str]=None)->pd.DataFrame:
        try:
            """
            export entire collectin as dataframe:
            return pd.DataFrame of collection
            """
            if database_name is None:
                collection = self.mongo_client.database[collection_name]
            else:
                collection = self.mongo_client[database_name][collection_name]

            df = pd.DataFrame(list(collection.find()))
            if "_id" in df.columns.to_list():
                df = df.drop(columns=["_id"], axis=1)
            df.replace({"na":np.nan},inplace=True)
            return df
        except Exception as e:
            raise BlackFridayException(e,sys)

In [ ]:
import sys

from black_friday.exception import BlackFridayException
from black_friday.logger import logging

import os
from black_friday.constants import DATABASE_NAME, MONGODB_URL_KEY
import pymongo
import certifi

ca = certifi.where()

class MongoDBClient:
    """
    Class Name :   export_data_into_feature_store
    Description :   This method exports the dataframe from mongodb feature store as dataframe 
    
    Output      :   connection to mongodb database
    On Failure  :   raises an exception
    """
    client = None

    def __init__(self, database_name=DATABASE_NAME) -> None:
        try:
            if MongoDBClient.client is None:
                mongo_db_url = os.getenv(MONGODB_URL_KEY)
                if mongo_db_url is None:
                    raise Exception(f"Environment key: {MONGODB_URL_KEY} is not set.")
                MongoDBClient.client = pymongo.MongoClient(mongo_db_url, tlsCAFile=ca)
            self.client = MongoDBClient.client
            self.database = self.client[database_name]
            self.database_name = database_name
            logging.info("MongoDB connection succesfull")
        except Exception as e:
            raise BlackFridayException(e,sys)